hey, created two ways to chat with models/csv files. first is below one in which after running code a box will show up above. you can enter csv file name (add the csv file in this folder) like country_full.csv and chat with it. or press enter to skip csv and go to normal chat. 

1st way (slower but advanced) :-

In [ ]:
import requests
import pandas as pd

def call_groq_api(content):
    url = "http://localhost:8000/process"
    response = requests.post(url, json={"content": content})
    return response.json()

def read_and_summarize_csv(file_path):
    """Read CSV file and return a summary string."""
    try:
        df = pd.read_csv(file_path)
        summary = f"Data Summary:\n\n{df.describe(include='all')}\n\nFirst few rows:\n{df.head().to_string()}"
        return summary
    except Exception as e:
        return f"Error reading the CSV file: {str(e)}"

if __name__ == "__main__":
    conversation_history = []
    
    # Optionally, read and analyze CSV file at the start
    file_path = input("Enter the path to your CSV file (or press Enter to skip): ")
    if file_path:
        csv_summary = read_and_summarize_csv(file_path)
        print(f"CSV File Summary:\n{csv_summary}\n")
        
        # Add CSV summary to the conversation history
        conversation_history.append({"role": "user", "content": "Analyzing the provided CSV file data."})
        conversation_history.append({"role": "assistant", "content": csv_summary})
    
    while True:
        # Input from the user
        content = input("You: ")
        
        # Print user message
        print(f"You: {content}")
        
        # Add user input to the conversation history
        conversation_history.append({"role": "user", "content": content})
        
        # Create a full context for the API call including previous conversation
        full_context = "\n".join([f"{entry['role']}: {entry['content']}" for entry in conversation_history])
        
        # Call the Groq API with the full context
        response = call_groq_api(full_context)
        
        # Get the assistant's response
        assistant_response = response.get('response', 'No response from the API.')
        
        # Print the API response
        print(f"Groq API: {assistant_response}")
        
        # Update conversation history
        conversation_history.append({"role": "assistant", "content": assistant_response})

        # Optionally break the loop on a specific command
        if content.lower() in ["exit", "quit"]:
            break


2nd way :- (faster but single response csv chat)

In [ ]:
#faster but single response csv chat
import requests
import pandas as pd

def call_groq_api(content):
    url = "http://localhost:8000/process"
    response = requests.post(url, json={"content": content})  # Sending content as JSON
    return response.json()

if __name__ == "__main__":
    # Load your CSV file
    csv_file_path = "country_full.csv"  # Change to your CSV file path
    df = pd.read_csv(csv_file_path)

    # Example: Generate a summary of the DataFrame
    content = f"Analyze the following data:\n{df.head().to_string()}"  # Adjust as needed
    response = call_groq_api(content)

    # Print the formatted response
    if 'response' in response:
        print("Groq API Response:\n")
        print(response['response'])
    else:
        print("Error in response:", response)